## Weights Calculation
 Weight0: Every Gene weighted equally 
 
 Weight1: Ralitive Risk
 
 Weight2: Number of Hits
 
 Weight3: LofZ

# Load Files

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os
ProjDIR = "/home/jw3514/Work/ASD_Circuits_CellType/" # Change to your project directory
sys.path.insert(1, f'{ProjDIR}/src/')
from ASD_Circuits import *
from plot import *

try:
    os.chdir(f"{ProjDIR}/notebooks_mouse_str/")
    print(f"Current working directory: {os.getcwd()}")
except FileNotFoundError as e:
    print(f"Error: Could not change directory - {e}")
except Exception as e:
    print(f"Unexpected error: {e}")


HGNC, ENSID2Entrez, GeneSymbol2Entrez, Entrez2Symbol = LoadGeneINFO()

In [ ]:
# Load config file
with open("../config/config.yaml", "r") as f:
    config = yaml.safe_load(f)

expr_matrix_path = config["analysis_types"]["STR_ISH"]["expr_matrix"]
STR_BiasMat = pd.read_parquet(f"../{expr_matrix_path}")
Anno = STR2Region()

In [ ]:
STR_BiasMat

# Spark

In [ ]:
Spark_Meta_2stage = pd.read_excel("../dat/Genetics/41588_2022_1148_MOESM4_ESM.xlsx",
                           skiprows=2, sheet_name="Table S7")
Spark_Meta_2stage = Spark_Meta_2stage[Spark_Meta_2stage[
    "pDenovoWEST_Meta"]!="."]   
Spark_Meta_ExomeWide = Spark_Meta_2stage[Spark_Meta_2stage["pDenovoWEST_Meta"]<=1.3e-6]
Spark_Meta_ExomeWide.shape


In [ ]:
lof_sum = Spark_Meta_ExomeWide["AutismMerged_LoF"].sum()
dmis_revel_sum = Spark_Meta_ExomeWide["AutismMerged_Dmis_REVEL0.5"].sum()

print(f"Sum of AutismMerged_LoF: {lof_sum}")
print(f"Sum of AutismMerged_Dmis_REVEL0.5: {dmis_revel_sum}")

In [ ]:
# import yaml

bgmr_path = config["data_files"]["Denovo_mut_rate"]
BGMR = pd.read_csv(bgmr_path, sep=None, engine='python', index_col=0)

# Add 'entrez_id' to each gene according to GeneName and GeneSymbol2Entrez
if "GeneName" in BGMR.columns:
    BGMR["entrez_id"] = BGMR["GeneName"].map(GeneSymbol2Entrez)
    # Convert entrez_id to int, dropping NAs
    BGMR = BGMR[~BGMR["entrez_id"].isna()].copy()
    BGMR["entrez_id"] = BGMR["entrez_id"].astype(int)
    BGMR = BGMR.set_index("entrez_id")
else:
    print("Warning: 'GeneName' column not found in BGMR.")

# Optionally, display a preview for verification
BGMR.head()

In [ ]:
BGMR.to_csv("/home/jw3514/Work/Resources/BGMR.withEntrez.csv")

In [ ]:
BGMR.columns

In [ ]:
Agg_gene2None, Agg_gene_bgmr = SPARK_Gene_Weights(Spark_Meta_ExomeWide, BGMR, 
        out="../dat/Unionize_bias/Spark_Meta_EWS.GeneWeight.bgmr.csv")
ASD_STR_Bias_bgmr = MouseSTR_AvgZ_Weighted(STR_BiasMat, Agg_gene_bgmr)
len(Agg_gene_bgmr)

Agg_gene2None, Agg_gene2MutN = SPARK_Gene_Weights(Spark_Meta_ExomeWide, None, 
        out="../dat/Unionize_bias/Spark_Meta_EWS.GeneWeight.v2.csv")
ASD_STR_Bias = MouseSTR_AvgZ_Weighted(STR_BiasMat, Agg_gene2MutN)
len(Agg_gene2MutN)



In [ ]:

                                        #csv_fil = "../dat/Unionize_bias/Spark.EW.Z2.v2.csv")
#D_Agg_avgZ_RD = RegionDistributionsList(ASD_STR_Bias, topN=50)
#print(D_Agg_avgZ_RD)

In [ ]:
ASD_STR_Bias.head(50)

In [ ]:
ASD_Z2_old = pd.read_csv("../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.FDR.csv", index_col="STR")

In [ ]:
#BiasCorrelation(ASD_Z2, ASD_Z2_old)

In [ ]:
# old_weights = LoadGeneWeights("../dat/Unionize_bias/Spark_Meta_EWS.GeneWeight.csv")
# ASD_Z2_old2 = AvgSTRZ_Weighted(ExpZ2Mat, old_weights, Method = 1)
# ASD_Z2_old2 = ASD_Z2_old2.set_index("STR")

In [ ]:
BiasCorrelation(ASD_STR_Bias, ASD_Z2_old, name1="Original ASD STR Bias", name2="ASD STR Bias with \nMutation Rate correction", dpi=200)
#BiasCorrelation(ASD_STR_Bias, ASD_Z2_old2)

In [ ]:
def plot_structure_bias_correlation(df_a, df_b, label_a='Dataset A', label_b='Dataset B', title=None):
    """
    Create comparison plot between two structure bias datasets

    Parameters:
    df_a: DataFrame with EFFECT column for first dataset
    df_b: DataFrame with EFFECT column for second dataset
    label_a: Label for x-axis (first dataset)
    label_b: Label for y-axis (second dataset)
    title: Custom title for the plot (ignored, no title drawn)

    Returns:
    correlation: Pearson correlation coefficient
    """
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(1, 1, dpi=120, figsize=(5, 4), facecolor='none')

    fig.patch.set_alpha(0)
    ax.patch.set_alpha(0)

    # Merge the datasets on structure names for comparison
    merged_data = pd.merge(df_a[['EFFECT']], df_b[['EFFECT']], 
                          left_index=True, right_index=True, suffixes=('_A', '_B'))

    # Create scatter plot
    ax.scatter(merged_data['EFFECT_A'], merged_data['EFFECT_B'], 
              alpha=1, s=20, c='#1f77b4', edgecolors='black', linewidth=0.5)

    # Add diagonal line for reference
    min_val = min(merged_data['EFFECT_A'].min(), merged_data['EFFECT_B'].min())
    max_val = max(merged_data['EFFECT_A'].max(), merged_data['EFFECT_B'].max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.8, linewidth=2, label='y=x')

    # Calculate correlation and p-value
    correlation, pval = pearsonr(merged_data['EFFECT_A'], merged_data['EFFECT_B'])

    # Set labels (no title)
    ax.set_xlabel(f'{label_a}', fontsize=14)
    ax.set_ylabel(f'{label_b}', fontsize=14)

    # Format p-value display according to instructions
    if pval < 1e-10:
        p_disp = f"p <{1e-10:.0e}"
    else:
        p_disp = f"p ={pval:.2g}"

    # Put correlation and p-value inside plot as annotation
    ax.annotate(f'r = {correlation:.3f}\n{p_disp}',
                xy=(0.05, 0.95), xycoords='axes fraction',
                ha='left', va='top', fontsize=15,
                bbox=dict(boxstyle="round,pad=0.3", fc="w", ec="gray", alpha=0.8))

    # Add grid
    ax.grid(True, linestyle='--', alpha=0.7)

    # Add legend
    ax.legend(fontsize=12)

    # Make axes equal for better comparison
    ax.set_aspect('equal', adjustable='box')

    plt.tight_layout() 
    plt.show()

    return correlation
plot_structure_bias_correlation(ASD_Z2_old, ASD_STR_Bias_bgmr, label_a='Mutation Bias\nZhou et al. 61 ASD genes', label_b='Mutation Bias (Expected Correction)\nZhou et al. 61 ASD genes', title='Structure Bias Comparison: Spark vs Fu_ASD_72')
#plot_structure_bias_correlation(Spark_ASD_STR_Bias, ASD_Female, label_a='Mutation Bias \nZhou et al. 61 ASD genes', label_b='Neuro-to-Glia Ratio Normalized Bias\nZhou et al. 61 ASD genes', title='Structure Bias Comparison: Spark vs Fu_ASD_185')

## Mutation Bootstrap







In [ ]:
Spark_Meta_ExomeWide

In [ ]:
Spark_Meta_ExomeWide_Genes = Spark_Meta_ExomeWide[["GeneID", "EntrezID", "HGNC", "ExACpLI", "LOEUF", "AutismMerged_LoF", "AutismMerged_Dmis_REVEL0.5", "pDenovoWEST_Meta"]]

In [ ]:
Spark_Meta_ExomeWide_Genes

In [ ]:
def bootstrap_gene_mutations(
    df,
    n_boot=10,
    weighted=True,
    lof_col="AutismMerged_LoF",
    dmis_col="AutismMerged_Dmis_REVEL0.5",
    rng=None,
):
    """
    Bootstrap mutation counts at the mutation level, preserving gene identity
    and total mutation load. Supports weighted (mutation-rate) or uniform resampling.

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe with gene-level LOF and Dmis mutation counts.
    n_boot : int, optional
        Number of bootstrap replicates (default = 10).
    weighted : bool, optional
        If True, mutations are resampled with probability proportional to observed counts.
        If False, each gene has equal probability of receiving any mutation.
    lof_col, dmis_col : str
        Column names for LOF and Dmis counts.
    rng : np.random.Generator, optional
        Numpy random generator for reproducibility.

    Returns
    -------
    boot_DFs : list of pd.DataFrame
        List of bootstrapped dataframes with resampled mutation counts and 'bootstrap_iter' column.
    """
    if rng is None:
        rng = np.random.default_rng(42)

    n = len(df)
    boot_DFs = []

    # Ensure integer non-negative mutation counts
    lof = df[lof_col].astype(int).clip(lower=0)
    dmis = df[dmis_col].astype(int).clip(lower=0)

    total_lof = lof.sum()
    total_dmis = dmis.sum()

    # Probability vectors for mutation assignment
    if weighted:
        # Weighted by observed mutation burden per gene
        p_lof = lof / total_lof if total_lof > 0 else np.ones(n) / n
        p_dmis = dmis / total_dmis if total_dmis > 0 else np.ones(n) / n
    else:
        # Uniform: every gene equally likely
        p_lof = np.ones(n) / n
        p_dmis = np.ones(n) / n

    for i in range(1, n_boot + 1):
        # Draw total_lof mutation events, assign to genes
        new_lof_counts = np.bincount(
            rng.choice(n, size=total_lof, replace=True, p=p_lof),
            minlength=n
        )
        new_dmis_counts = np.bincount(
            rng.choice(n, size=total_dmis, replace=True, p=p_dmis),
            minlength=n
        )

        # Create bootstrap replicate
        df_boot = df.copy().reset_index(drop=True)
        df_boot[lof_col] = new_lof_counts
        df_boot[dmis_col] = new_dmis_counts
        df_boot["bootstrap_iter"] = i
        df_boot["bootstrap_type"] = "weighted" if weighted else "uniform"
        boot_DFs.append(df_boot)

    return boot_DFs

In [ ]:
boot_DFs_weights = bootstrap_gene_mutations(Spark_Meta_ExomeWide_Genes, 1000, weighted=True)
boot_DFs_uniform = bootstrap_gene_mutations(Spark_Meta_ExomeWide_Genes, 1000, weighted=False)

In [ ]:
print(Spark_Meta_ExomeWide_Genes["AutismMerged_LoF"].sum(), Spark_Meta_ExomeWide_Genes["AutismMerged_Dmis_REVEL0.5"].sum())
Spark_Meta_ExomeWide_Genes

In [ ]:
idx = 1
print(boot_DFs_weights[idx]["AutismMerged_LoF"].sum(), boot_DFs_uniform[idx]["AutismMerged_Dmis_REVEL0.5"].sum())
boot_DFs_weights[idx]

In [ ]:
boot_bias_list_weights = []
save_dir = "../results/Bootstrap_bias/Spark_ExomeWide/Weighted_Resampling"
os.makedirs(save_dir, exist_ok=True)

for i, DF in enumerate(boot_DFs_weights):
    _, boot_gw = SPARK_Gene_Weights(DF, BGMR, Bmis=False)
    boot_bias = MouseSTR_AvgZ_Weighted(STR_BiasMat, boot_gw)
    boot_bias_list_weights.append(boot_bias)
    boot_bias.to_csv(os.path.join(save_dir, f"Spark_ExomeWide.GeneWeight.boot{i}.csv"))

In [ ]:
boot_bias_list_uniform = []
save_dir = "../results/Bootstrap_bias/Spark_ExomeWide/Uniform_Resampling"
os.makedirs(save_dir, exist_ok=True)

for i, DF in enumerate(boot_DFs_uniform):
    _, boot_gw = SPARK_Gene_Weights(DF, BGMR)
    boot_bias = MouseSTR_AvgZ_Weighted(STR_BiasMat, boot_gw)
    boot_bias_list_uniform.append(boot_bias)
    boot_bias.to_csv(os.path.join(save_dir, f"Spark_ExomeWide.GeneWeight.boot{i}.csv"))

In [ ]:
boot_bias_list_weights

In [ ]:
ScoreMatDir="/home/jw3514/Work/ASD_Circuits/dat/allen-mouse-conn/ScoreingMat_jw_v3/"
IpsiInfoMat=pd.read_csv(ScoreMatDir + "InfoMat.Ipsi.csv", index_col=0)
IpsiInfoMatShort_v1=pd.read_csv(ScoreMatDir + "InfoMat.Ipsi.Short.3900.csv", index_col=0)
IpsiInfoMatLong_v1=pd.read_csv(ScoreMatDir + "InfoMat.Ipsi.Long.3900.csv", index_col=0)

DIR = "/home/jw3514/Work/ASD_Circuits/scripts/RankScores/"
Cont_Distance = np.load("{}/RankScore.Ipsi.Cont.npy".format(DIR))
Cont_DistanceShort = np.load("{}/RankScore.Ipsi.Short.3900.Cont.npy".format(DIR))
Cont_DistanceLong = np.load("{}/RankScore.Ipsi.Long.3900.Cont.npy".format(DIR))

In [ ]:
from collections import defaultdict

def compute_circuit_scores_for_profiles(
    profile_bias_dict,
    topNs,
    info_mats_dict,
    scoring_func=ScoreCircuit_SI_Joint,
):
    """
    Compute circuit scores for multiple profiles and connection types.

    Args:
        profile_bias_dict: dict of {profile_name: STR_Bias DataFrame}
        topNs: list of top N structure ranks to scan
        info_mats_dict: dict of {conn_type: info_mat pandas DataFrame}
        scoring_func: function (top_str_list, info_mat) => score

    Returns:
        results: dict of {profile_name: {conn_type: np.array of scores}}
    """
    results = {profile_name: {conn_type: [] for conn_type in info_mats_dict}
               for profile_name in profile_bias_dict}
    for profile_name, bias_df in profile_bias_dict.items():
        str_ranks = bias_df.sort_values("EFFECT", ascending=False).index.values
        for topN in topNs:
            top_strs = str_ranks[:topN]
            for conn_type, info_mat in info_mats_dict.items():
                score = scoring_func(top_strs, info_mat)
                results[profile_name][conn_type].append(score)
    # Convert lists to np.arrays for easier plotting
    for profile_name in results:
        for conn_type in results[profile_name]:
            results[profile_name][conn_type] = np.array(results[profile_name][conn_type])
    return results

def plot_circuit_connectivity_scores_multi(
    topNs,
    circuit_scores_results,
    cont_distance_dict,
    profile_plot_kwargs=None,
    show_siblings=True,
    profile_labels=None,
    xlim=(0, 121)
):
    """
    Plot circuit connectivity scores for multiple profiles and connection types.
    - circuit_scores_results: output of compute_circuit_scores_for_profiles
    - cont_distance_dict: {conn_type: np.array [n_iter, len(topNs)]}
    - profile_plot_kwargs: {profile_name: {kwargs for plt.plot}}
    - profile_labels: {profile_name: label string}
    """
    conn_types = list(cont_distance_dict.keys())
    n_conn = len(conn_types)
    fig, axes = plt.subplots(n_conn, 1, dpi=480, figsize=(7, 4*n_conn))

    if n_conn == 1:
        axes = [axes]

    BarLen = 34.1

    colors = ["blue", "red", "purple", "orange", "green", "black", "brown"]
    if profile_plot_kwargs is None:
        profile_plot_kwargs = {}
    if profile_labels is None:
        profile_labels = {}

    for i, conn_type in enumerate(conn_types):
        ax = axes[i]
        cont = (np.median if not conn_type.lower().startswith("long") else np.nanmean)(cont_distance_dict[conn_type], axis=0)
        # Plot scores for all profiles
        for idx, (profile_name, prof_scores) in enumerate(circuit_scores_results.items()):
            scores = prof_scores[conn_type]
            label = profile_labels.get(profile_name, profile_name)
            plot_args = profile_plot_kwargs.get(profile_name, {})
            if not plot_args:
                # Generate plot styles dynamically
                plot_args = dict(
                    color=colors[idx % len(colors)],
                    marker=["o", "s", "^", "d", "x", "v"][idx % 6],
                    markersize=5 if idx == 0 else 3,
                    lw=1,
                    ls='dashed' if idx == 0 else '-',
                    label=label
                )
            ax.plot(topNs, scores, **plot_args)

        # Plot Sibling controls
        if show_siblings:
            #if "nan" in str(type(cont_distance_dict[conn_type])):
            lower = np.nanpercentile(cont_distance_dict[conn_type], 50-BarLen, axis=0)
            upper = np.nanpercentile(cont_distance_dict[conn_type], 50+BarLen, axis=0)
            #else:
            #    lower = np.percentile(cont_distance_dict[conn_type], 50-BarLen, axis=0)
            #    upper = np.percentile(cont_distance_dict[conn_type], 50+BarLen, axis=0)
            ax.errorbar(
                topNs, cont, color="grey", marker="o", markersize=1.5, lw=1,
                yerr=(cont - lower, np.abs(upper - cont)),
                ls="dashed", label="Siblings"
            )

        ax.set_xlabel("Structure Rank\n", fontsize=17)
        ax.set_ylabel("Circuit Connectivity Score", fontsize=15)
        ax.legend(fontsize=13)
        ax.set_xlim(*xlim)
        ax.grid(True)
        ax.set_title(f'Connection Type: {conn_type}', fontsize=15)
    plt.tight_layout()
    return fig

# -- USAGE EXAMPLE/INSTANTIATION --

topNs = list(range(200, 5, -1))

# Define profiles to use (easily reusable/extendable!)
profiles = {
    "Spark 61": ASD_STR_Bias,
    "Spark 61 Old": ASD_Z2_old,
    # "Spark": Spark_ASD_STR_Bias, # add more as needed
}
info_mats = {
    "Standard": IpsiInfoMat,
    "Short": IpsiInfoMatShort_v1,
    "Long": IpsiInfoMatLong_v1,
}
cont_distance_dict = {
    "Standard": Cont_Distance,
    "Short": Cont_DistanceShort,
    "Long": Cont_DistanceLong,
}

# Compute all
circuit_scores = compute_circuit_scores_for_profiles(
    profiles, topNs, info_mats
)

# Plot
fig = plot_circuit_connectivity_scores_multi(
    topNs,
    circuit_scores,
    cont_distance_dict,
    profile_labels={
        "Fu_ASD_185": "185 genes (Fu_ASD_185)",
        "Fu_ASD_72": "72 genes (Fu_ASD_72)",
    },
    xlim=(0, 121)
)
plt.show()

## Bootstrap Confidence Intervals for Circuit Connectivity Scores

Now we compute the circuit connectivity scores for each bootstrap replicate to generate confidence intervals.


In [ ]:
# Compute circuit scores for all bootstrap samples
topNs = list(range(200, 5, -1))
info_mats = {
    "Standard": IpsiInfoMat,
    "Short": IpsiInfoMatShort_v1,
    "Long": IpsiInfoMatLong_v1,
}

# Initialize storage for bootstrap scores
boot_circuit_scores_weights = {conn_type: [] for conn_type in info_mats.keys()}
boot_circuit_scores_uniform = {conn_type: [] for conn_type in info_mats.keys()}

print(f"Computing circuit scores for {len(boot_bias_list_weights)} bootstrap samples...")
for boot_idx, (boot_bias, boot_bias_uniform) in enumerate(zip(boot_bias_list_weights, boot_bias_list_uniform)):
    if (boot_idx + 1) % 20 == 0:
        print(f"  Processing bootstrap {boot_idx + 1}/{len(boot_bias_list_weights)}")
    
    # Get structure ranks for this bootstrap (weighted and uniform)
    str_ranks = boot_bias.sort_values("EFFECT", ascending=False).index.values
    str_ranks_uniform = boot_bias_uniform.sort_values("EFFECT", ascending=False).index.values
    
    # Compute scores for each topN and connection type for weighted
    for conn_type, info_mat in info_mats.items():
        scores_this_boot = []
        for topN in topNs:
            top_strs = str_ranks[:topN]
            score = ScoreCircuit_SI_Joint(top_strs, info_mat)
            scores_this_boot.append(score)
        boot_circuit_scores_weights[conn_type].append(scores_this_boot)
        
    # Compute scores for each topN and connection type for uniform
    for conn_type, info_mat in info_mats.items():
        scores_this_boot_uniform = []
        for topN in topNs:
            top_strs_uniform = str_ranks_uniform[:topN]
            score_uniform = ScoreCircuit_SI_Joint(top_strs_uniform, info_mat)
            scores_this_boot_uniform.append(score_uniform)
        boot_circuit_scores_uniform[conn_type].append(scores_this_boot_uniform)

# Convert to numpy arrays: shape (n_bootstrap, n_topNs)
for conn_type in boot_circuit_scores_weights:
    boot_circuit_scores_weights[conn_type] = np.array(boot_circuit_scores_weights[conn_type])
    print(f"{conn_type} (weighted): shape = {boot_circuit_scores_weights[conn_type].shape}")

for conn_type in boot_circuit_scores_uniform:
    boot_circuit_scores_uniform[conn_type] = np.array(boot_circuit_scores_uniform[conn_type])
    print(f"{conn_type} (uniform): shape = {boot_circuit_scores_uniform[conn_type].shape}")

print("Done!")


In [ ]:
def plot_circuit_scores_with_bootstrap_ci(
    topNs,
    mean_scores,
    boot_scores,
    cont_distance_dict,
    ci_type='percentile',  # 'percentile' for 95% CI or 'std' for 1 std
    percentile_range=95,   # percentile range (e.g., 95 for 95% CI)
    viz_style='shade',     # 'shade' for shaded region or 'errorbar'
    show_siblings=True,
    xlim=(0, 121),
    colors=None,
    labels=None
):
    """
    Plot circuit connectivity scores with bootstrap confidence intervals.
    
    Parameters:
    -----------
    topNs : list
        List of top N structure ranks
    mean_scores : dict
        Dictionary {conn_type: mean_scores_array}
    boot_scores : dict
        Dictionary {conn_type: bootstrap_scores_array (n_boot, n_topNs)}
    cont_distance_dict : dict
        Dictionary {conn_type: control_distance_array}
    ci_type : str
        'percentile' for percentile-based CI (default 95%)
        'std' for mean ± 1 standard deviation
    percentile_range : float
        Percentile range for CI when ci_type='percentile' (default 95)
    viz_style : str
        'shade' for shaded region (default)
        'errorbar' for error bars
    show_siblings : bool
        Whether to show sibling controls
    xlim : tuple
        X-axis limits
    colors : dict
        Custom colors for each connection type
    labels : dict
        Custom labels for each connection type
        
    Returns:
    --------
    fig : matplotlib figure
    """
    conn_types = list(mean_scores.keys())
    n_conn = len(conn_types)
    
    # Tighter figure layout
    fig, axes = plt.subplots(n_conn, 1, dpi=480, figsize=(7, 2.5*n_conn), 
                             sharex=True)
    
    if n_conn == 1:
        axes = [axes]
    
    BarLen = 34.1
    
    default_colors = {
        "Standard": "blue",
        "Short": "red", 
        "Long": "purple"
    }
    default_labels = {
        "Standard": "Full Connectome",
        "Short": "Short Distance\nConnectome",
        "Long": "Long Distance\nConnectome"
    }
    
    if colors is None:
        colors = default_colors
    if labels is None:
        labels = default_labels
    
    # Collect all y-values for aligned y-limits
    all_y_values = []
    
    for i, conn_type in enumerate(conn_types):
        ax = axes[i]
        
        # Get mean and bootstrap scores
        mean_score = mean_scores[conn_type]
        boot_score = boot_scores[conn_type]  # shape: (n_boot, n_topNs)
        
        # Calculate confidence intervals (ignoring NaN values in bootstrap)
        if ci_type == 'percentile':
            lower_percentile = (100 - percentile_range) / 2
            upper_percentile = 100 - lower_percentile
            lower_ci = np.nanpercentile(boot_score, lower_percentile, axis=0)
            upper_ci = np.nanpercentile(boot_score, upper_percentile, axis=0)
            ci_label = f'{percentile_range}% CI'
        elif ci_type == 'std':
            std = np.nanstd(boot_score, axis=0)
            lower_ci = mean_score - std
            upper_ci = mean_score + std
            ci_label = '±1 std'
        else:
            raise ValueError(f"Unknown ci_type: {ci_type}")
        
        # Collect y values for consistent scaling
        all_y_values.extend(mean_score[~np.isnan(mean_score)])
        all_y_values.extend(lower_ci[~np.isnan(lower_ci)])
        all_y_values.extend(upper_ci[~np.isnan(upper_ci)])
        
        # Plot mean line
        color = colors.get(conn_type, "blue")
        label = labels.get(conn_type, conn_type)
        ax.plot(topNs, mean_score, color=color, marker='o', markersize=3, 
                lw=1.5, label=f'{label}', zorder=3)
        
        # Plot confidence intervals
        if viz_style == 'shade':
            ax.fill_between(topNs, lower_ci, upper_ci, color=color, 
                           alpha=0.2, label=ci_label, zorder=2)
        elif viz_style == 'errorbar':
            # Sample every Nth point to avoid overcrowding
            step = max(1, len(topNs) // 20)
            indices = range(0, len(topNs), step)
            ax.errorbar(
                [topNs[j] for j in indices],
                [mean_score[j] for j in indices],
                yerr=[
                    [mean_score[j] - lower_ci[j] for j in indices],
                    [upper_ci[j] - mean_score[j] for j in indices]
                ],
                fmt='none', ecolor=color, alpha=0.5, capsize=2,
                label=ci_label, zorder=2
            )
        else:
            raise ValueError(f"Unknown viz_style: {viz_style}")
        
        # Plot sibling controls
        if show_siblings:
            cont = (np.median if not conn_type.lower().startswith("long") 
                   else np.nanmean)(cont_distance_dict[conn_type], axis=0)
            lower_sib = np.nanpercentile(cont_distance_dict[conn_type], 50-BarLen, axis=0)
            upper_sib = np.nanpercentile(cont_distance_dict[conn_type], 50+BarLen, axis=0)
            ax.errorbar(
                topNs, cont, color="grey", marker="o", markersize=1.5, lw=1,
                yerr=(cont - lower_sib, np.abs(upper_sib - cont)),
                ls="dashed", label="Siblings", zorder=1
            )
            all_y_values.extend(lower_sib[~np.isnan(lower_sib)])
            all_y_values.extend(upper_sib[~np.isnan(upper_sib)])
        
        # Only show xlabel and ylabel on bottom subplot
        if i == n_conn - 1:
            ax.set_xlabel("Structure Rank", fontsize=25)
            ax.set_ylabel("Circuit Connectivity Score", fontsize=20)
        
        ax.legend(fontsize=11, loc='upper right', frameon=True, framealpha=0.9)
        ax.set_xlim(*xlim)
        ax.grid(True, alpha=0.3)
        ax.tick_params(axis='both', labelsize=18)
    
    # Set consistent y-limits across all subplots for alignment
    if all_y_values:
        y_min = np.min(all_y_values)
        y_max = np.max(all_y_values)
        y_range = y_max - y_min
        y_padding = y_range * 0.05
        for ax in axes:
            ax.set_ylim(y_min - y_padding, y_max + y_padding)
    
    plt.tight_layout(h_pad=0.5)
    return fig


In [ ]:
# Compute mean scores from the original Spark 61 data
mean_circuit_scores = {}

str_ranks = ASD_STR_Bias.sort_values("EFFECT", ascending=False).index.values

for conn_type, info_mat in info_mats.items():
    scores = []
    for topN in topNs:
        top_strs = str_ranks[:topN]
        score = ScoreCircuit_SI_Joint(top_strs, info_mat)
        scores.append(score)
    mean_circuit_scores[conn_type] = np.array(scores)

print("Mean circuit scores computed for all connection types.")


### Plot 1: 95% Confidence Intervals (Shaded Region) - DEFAULT


In [ ]:
# Plot with 95% CI and shaded regions (DEFAULT)
fig1 = plot_circuit_scores_with_bootstrap_ci(
    topNs=topNs,
    mean_scores=mean_circuit_scores,
    boot_scores=boot_circuit_scores_weights,
    cont_distance_dict=cont_distance_dict,
    ci_type='percentile',      # Use percentile-based CI
    percentile_range=95,        # 95% CI
    viz_style='shade',          # Shaded region
    show_siblings=True,
    xlim=(0, 121)
)
plt.show()


In [ ]:
def plot_circuit_scores_with_bootstrap_ci(
    topNs,
    mean_scores,
    boot_scores,
    cont_distance_dict,
    ci_type='percentile',      # 'percentile' for 95% CI or 'std' for 1 std
    percentile_range=95,       # percentile range (e.g., 95 for 95% CI)
    viz_style='shade',         # deprecated, now controlled by show_asd_ci and show_sib_ci
    show_siblings=True,        # deprecated, will show sib curve if show_sib_ci or both
    xlim=(0, 121),
    colors=None,
    labels=None,
    show_asd_ci=True,          # NEW: show ASD bootstrap shade/error
    show_sib_ci=True           # NEW: show siblings errorbar
):
    """
    Plot circuit connectivity scores with bootstrap confidence intervals and/or sibling error bars.

    Parameters:
    -----------
    topNs : list
        List of top N structure ranks
    mean_scores : dict
        Dictionary {conn_type: mean_scores_array}
    boot_scores : dict
        Dictionary {conn_type: bootstrap_scores_array (n_boot, n_topNs)}
    cont_distance_dict : dict
        Dictionary {conn_type: control_distance_array}
    ci_type : str
        'percentile' for percentile-based CI (default 95%)
        'std' for mean ± 1 standard deviation
    percentile_range : float
        Percentile range for CI when ci_type='percentile' (default 95)
    viz_style : str
        'shade' for shaded region (default) [deprecated]
        'errorbar' for error bars         [deprecated]
    show_siblings : bool
        (deprecated, use show_sib_ci)
    xlim : tuple
        X-axis limits
    colors : dict
        Custom colors for each connection type
    labels : dict
        Custom labels for each connection type
    show_asd_ci : bool
        Whether to show ASD bootstrap CI area/bar (shaded/error bar)
    show_sib_ci : bool
        Whether to show sibling error bars

    Returns:
    --------
    fig : matplotlib figure
    """
    conn_types = list(mean_scores.keys())
    n_conn = len(conn_types)
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, axes = plt.subplots(n_conn, 1, dpi=480, figsize=(8, 4*n_conn))

    if n_conn == 1:
        axes = [axes]

    BarLen = 34.1

    default_colors = {
        "Standard": "blue",
        "Short": "red", 
        "Long": "purple"
    }
    default_labels = {
        "Standard": "ASD Probands\n(Full Connectome)",
        "Short": "ASD Probands\n(Short Distance Connectome)",
        "Long": "ASD Probands\n(Long Distance Connectome)"
    }

    if colors is None:
        colors = default_colors
    if labels is None:
        labels = default_labels

    for i, conn_type in enumerate(conn_types):
        ax = axes[i]

        # ASD: Mean and bootstrap scores
        mean_score = mean_scores[conn_type]
        boot_score = boot_scores[conn_type]  # shape: (n_boot, n_topNs)

        # Calculate confidence intervals (ignoring NaN values in bootstrap)
        if ci_type == 'percentile':
            lower_percentile = (100 - percentile_range) / 2
            upper_percentile = 100 - lower_percentile
            lower_ci = np.nanpercentile(boot_score, lower_percentile, axis=0)
            upper_ci = np.nanpercentile(boot_score, upper_percentile, axis=0)
            ci_label = f'ASD Probands \n{percentile_range}% CI'
        elif ci_type == 'std':
            std = np.nanstd(boot_score, axis=0)
            lower_ci = mean_score - std
            upper_ci = mean_score + std
            ci_label = '±1 std'
        else:
            raise ValueError(f"Unknown ci_type: {ci_type}")

        # Plot ASD mean line (ALWAYS SHOWN)
        color = colors.get(conn_type, "blue")
        label = labels.get(conn_type, conn_type)
        ax.plot(topNs, mean_score, color=color, marker='o', markersize=5, 
                lw=1.5, label=f'{label}', zorder=3)

        # Plot ASD confidence interval (shade or errorbar)
        if show_asd_ci:
            if viz_style == 'shade':
                ax.fill_between(topNs, lower_ci, upper_ci, color=color, 
                                alpha=0.2, label=ci_label, zorder=2)
            elif viz_style == 'errorbar':
                step = max(1, len(topNs) // 20)
                indices = range(0, len(topNs), step)
                ax.errorbar(
                    [topNs[j] for j in indices],
                    [mean_score[j] for j in indices],
                    yerr=[
                        [mean_score[j] - lower_ci[j] for j in indices],
                        [upper_ci[j] - mean_score[j] for j in indices]
                    ],
                    fmt='none', ecolor=color, alpha=0.5, capsize=2,
                    label=ci_label, zorder=2
                )
            else:
                raise ValueError(f"Unknown viz_style: {viz_style}")

        # Plot siblings: Always plot average sibling profile (grey solid line), 
        # add error bar only if show_sib_ci=True.
        cont_func = np.median if not conn_type.lower().startswith("long") else np.nanmean
        cont_profile = cont_func(cont_distance_dict[conn_type], axis=0)
        ax.plot(topNs, cont_profile, color="grey", marker="o", markersize=3, lw=1.5,
                label="Siblings", zorder=4)

        if show_sib_ci:
            lower = np.nanpercentile(cont_distance_dict[conn_type], 50-BarLen, axis=0)
            upper = np.nanpercentile(cont_distance_dict[conn_type], 50+BarLen, axis=0)
            ax.errorbar(
                topNs, cont_profile, color="grey", marker="o", markersize=1.5, lw=1,
                yerr=(cont_profile - lower, np.abs(upper - cont_profile)),
                ls="dashed", label="Siblings (Interquantile Range)", zorder=1,
                alpha=0.77
            )

        # Remove x label and title for all but the bottom-most subplot
        if i == n_conn - 1:
            ax.set_xlabel("Structure Rank\n", fontsize=25)
        else:
            ax.set_xlabel("")
        ax.set_ylabel("Circuit Connectivity Score", fontsize=20, weight='normal')
        ax.legend(fontsize=13, loc='upper right', frameon=False)
        ax.set_xlim(*xlim)
        ax.grid(True, alpha=0.3)
        # Only set title for single-plot case (optional, comment if unused)
        # if n_conn == 1:
        #    ax.set_title(f'Connection Type: {conn_type}', fontsize=15)
        # No per-subplot title

    plt.tight_layout()
    return fig

# Example: ASD+shade+error, sibling+error
fig1 = plot_circuit_scores_with_bootstrap_ci(
    topNs=topNs,
    mean_scores=mean_circuit_scores,
    boot_scores=boot_circuit_scores_weights,
    cont_distance_dict=cont_distance_dict,
    ci_type='percentile',      # Use percentile-based CI
    percentile_range=95,       # 95% CI
    viz_style='shade',         # Shaded region for ASD
    show_asd_ci=False,          # Show ASD bootstrap shade or errorbar
    show_sib_ci=True,          # Show sibling errorbar as well
    xlim=(0, 121)
)
plt.show()

In [ ]:
# Example: ASD+shade+error, sibling+error
fig1 = plot_circuit_scores_with_bootstrap_ci(
    topNs=topNs,
    mean_scores=mean_circuit_scores,
    boot_scores=boot_circuit_scores_weights,
    cont_distance_dict=cont_distance_dict,
    ci_type='percentile',      # Use percentile-based CI
    percentile_range=95,       # 95% CI
    viz_style='shade',         # Shaded region for ASD
    show_asd_ci=True,          # Show ASD bootstrap shade or errorbar
    show_sib_ci=False,          # Show sibling errorbar as well
    xlim=(0, 121)
)
plt.show()